## RAG Day 3

### Expert Question Answerer for InsureLLM

LangChain 1.0 implementation of a RAG pipeline.

Using the VectorStore we created last time (with HuggingFace `all-MiniLM-L6-v2`)

In [23]:
# Fix ALL missing attributes from LangChain 0.3.x (ChatOpenAI + retrievers need these)
import langchain

# Add missing attributes if they don't exist
if not hasattr(langchain, "verbose"):
    langchain.verbose = False

if not hasattr(langchain, "debug"):
    langchain.debug = False

if not hasattr(langchain, "llm_cache"):
    langchain.llm_cache = None

print("Patched langchain.verbose, langchain.debug, langchain.llm_cache")


Patched langchain.verbose, langchain.debug, langchain.llm_cache


In [24]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings
import gradio as gr

In [26]:
MODEL = "moonshotai/kimi-k2-instruct-0905"
DB_NAME = "vector_db"
load_dotenv(override=True)

True

### Connect to Chroma; use Hugging Face all-MiniLM-L6-v2

In [27]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)

### Set up the 2 key LangChain objects: retriever and llm

#### A sidebar on "temperature":
- Controls how diverse the output is
- A temperature of 0 means that the output should be predictable
- Higher temperature for more variety in answers

Some people describe temperature as being like 'creativity' but that's not quite right
- It actually controls which tokens get selected during inference
- temperature=0 means: always select the token with highest probability
- temperature=1 usually means: a token with 10% probability should be picked 10% of the time

Note: a temperature of 0 doesn't mean outputs will always be reproducible. You also need to set a random seed. We will do that in weeks 6-8. (Even then, it's not always reproducible.)

Note 2: if you want creativity, use the System Prompt!

In [55]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=MODEL,
    temperature=0.7
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}     # or any RETRIEVAL_K
)


### These LangChain objects implement the method `invoke()`

In [45]:
import langchain

# PATCH missing attributes due to LangChain 0.3.x bugs
langchain.verbose = False
langchain.debug = False


In [46]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are Ayush Tyagi's personal AI assistant. Use the retrieved context to answer."),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

output_parser = StrOutputParser()

def rag_chain(question):
    # retrieve documents
    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([d.page_content for d in docs])

    # run prompt + LLM
    response = llm.invoke(prompt.format(
        question=question,
        context=context
    ))

    return response


In [47]:
# CELL: Patch LangChain global variables (fixes debug + verbose error)
import langchain

if not hasattr(langchain, "verbose"):
    langchain.verbose = False

if not hasattr(langchain, "debug"):
    langchain.debug = False

print("LangChain patches applied.")


LangChain patches applied.


In [48]:
answer = rag_chain("Who is Ayush Tyagi?")
print(answer)


content="Ayush Tyagi is a Computer Science Engineering student at JIMS, Greater Noida (IP University) set to graduate in 2025. He specializes in front-end development, creating visually stunning and user-friendly web applications. \n\nHis background includes:\n- Education: PCM stream at Vivekanand School, Anand Vihar (2021) followed by B.Tech CSE at JIMS Greater Noida\n- Technical skills: Front-end development, Python, Unity Engine, C#, and AI frameworks\n- Professional experience: Game Development Intern at Tara Application, where he worked on mobile game reskinning, monetization, and publishing\n- Notable projects: Built a multimodal AI assistant combining image generation and personalized chat capabilities\n\nHe's currently available for internships and collaborative projects, with a focus on blending creative design with technical execution to deliver exceptional digital experiences. You can reach him at tyagiayush239@gmail.com or through his LinkedIn profile." additional_kwargs={'

In [49]:
answer = rag_chain("Who is vivekanand?")
print(answer)

content='Vivekanand (formally “Swami Vivekanand”) was a 19th-century Indian monk, philosopher, and social reformer who inspired the name of Vivekanand School, Anand Vihar—the school from which Ayush Tyagi completed his 12th-grade PCM education in 2021.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 731, 'total_tokens': 795, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'reasoning_tokens': 0}, 'model_name': 'moonshotai/kimi-k2-instruct-0905', 'system_fingerprint': None, 'id': '60177d1c4cf94777aac694a05a211883', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--ce89477c-0d9e-4681-8baa-bb5e90e0014d-0' usage_metadata={'input_tokens': 731, 'output_tokens': 64, 'total_tokens': 795, 'input_token_details': {}, 'output_token_details': {}}


## Time to put this together!

In [50]:
SYSTEM_PROMPT_TEMPLATE = """
You are Ayush Tyagi’s personal AI assistant. 
and give your intro when someone greets you.
Your job is to answer questions using the provided context and your general knowledge.

Follow this skill hierarchy when responding:
1. Prioritize knowledge related to LLMs, vector databases, embeddings, RAG, AI engineering, and backend integrations.
2. Next, prioritize Ayush’s expertise in AI tools, machine learning workflows, automation, and software engineering.
3. Then focus on frontend development, especially React, React Native, Tailwind, Next.js, animations, UI/UX, and component-level design.
4. After that, cover Ayush’s experience in game development, Unity, Android Studio, and other technical domains.
5. For topics outside these areas, answer normally—clearly and helpfully.

Rules:
- Always stay friendly, supportive, and knowledgeable.
- If the question relates to Ayush’s skills, answer with authority.
- If you don’t know something or the context doesn't include it, say so.
- Use the context below whenever it contains relevant information.
- Never invent facts.

Context:
{context}
"""


In [51]:
def answer_question(question: str, history):
    docs = retriever.invoke(question) #searches your vector database.Jo question se related documents hote hain, woh laata hai.
    context = "\n\n".join(doc.page_content for doc in docs) #CONTEXT THAT WE WILL GIVE TO OUR MODEL BY COMBINIG IT 
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    return response.content

In [53]:
answer_question("linkdin of aayush?", []) #i.e fuzzy matching because of vd and rags 
#openAi models maybe way better 

"Hey! I'm Ayush's AI assistant—here to help with anything about his work, skills, or projects.  \n\nLinkedIn: https://www.linkedin.com/in/ayush-tyagi-0a3694267"

## What could possibly come next? 😂

In [43]:
gr.ChatInterface(answer_question).launch()

c:\Users\ayush\AppData\Local\anaconda3\envs\llms\Lib\site-packages\gradio\chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## Admit it - you thought RAG would be more complicated than that!!